In [1]:
import numpy as np
import sys
sys.path.append('..')
sys.path.append('../analyses')

import analysis
import datatypes
import qm9_filter_generated

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [4]:
methane = analysis.construct_molecule('CH3')[0]
valence = np.array([0, 1, 0, 0, 0, 0, 4, 3, 2, 1])

In [5]:
qm9_filter_generated.check_valency(np.asarray([methane.positions]), np.asarray([list(methane.numbers)]), valence)

{'mols': [<utility_classes.Molecule at 0x107957850>],
 'connectivity': array([[[0., 1., 1., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]]]),
 'valid_mol': array([False]),
 'valid_atom': array([0.75])}